In [1]:
import numpy as np
import csv

# 提取用户-电影矩阵
def user_movie_matrix(filename):
    with open(filename,'r') as f:
        reader = csv.reader(f)
        # 668用户，149532部电影（有缺失数据）
        # id对应行列数
        res=np.zeros([669,149533])
        for item in reader:
            # 忽略第一行
            if reader.line_num == 1:
                continue
            res[int(item[0])][int(item[1])]=float(item[2])
        return res


# 记录每位用户观看的电影id
def userWatched(rating_martix):
    res={}
    for user in range(len(rating_martix)):
        res[user] =[]
        for movie in range(len(rating_martix[0])):
            if rating_martix[user][movie]!=0:
                res[user].append(movie)
    return res

# 提取某部电影的流派信息
# name 收录所有电影名称{id:move}
# idx 对应电影ID
# dic 为电影和其对应流派的字典
def loadGenre(filename):
    dm=open(filename,'r')
    reader=csv.reader(dm)
    mov_gen,name,gen_mov={},{},{}
    for item in reader:
        # 忽略第一行
        if reader.line_num == 1:
            continue
        name[item[0]]=item[1]
        genre=item[2].split('|')
        mov_gen[item[1]]=genre
        # 将流派和电影对应
        for gen in genre:
            if not gen in gen_mov:
                gen_mov[gen]=[item[1]]
            else:
                gen_mov[gen].append(item[1])
    return name,mov_gen,gen_mov


# 返回{电影名称:[总分，[评分人id]]}
# 返回{用户id:[[电影id],[评分]]}
def loadRating(filename,name):
    dm=open(filename,'r')
    reader=csv.reader(dm)
    rating,user={},{}
    for item in reader:
        # 忽略第一行
        if reader.line_num == 1:
            continue
        if not item[0] in user:
            user[item[0]]=[[item[1]],[float(item[2])]]
        else:
            user[item[0]][0].append(item[1])
            user[item[0]][1].append(float(item[2]))

        if not name[item[1]] in rating:
            # [目前总分，[评分人id]]
            rating[name[item[1]]]=[float(item[2]),[item[0]]]
        else:
            rating[name[item[1]]][0]+=float(item[2])
            rating[name[item[1]]][1].append(item[0])
    return rating,user

# 分析各个流派平均分
def rateGenre(genre_mov,rating):
    gen_rating={}
    # 遍历每个流派
    for gen in genre_mov:
        if not gen in gen_rating:
            gen_rating[gen]=0
        # 遍历这个流派下的电影
        score=0
        # 这个流派下电影的数
        num = len(genre_mov[gen])
        for mov in genre_mov[gen]:
            if mov in rating:
                score+=rating[mov][0]/len(rating[mov][1])
        gen_rating[gen]=score/num
    return gen_rating

# 返回每个用户对每个流派的平均评分
# 如没看过则为0分
# {userID:[3,4,2,0,3,...]}
# 序号同genre列表
def user_genre(user,mov_gen,gen_mov,name):
    # genre列表
    gen=[]
    for i in gen_mov:
        gen.append(i)
    user_gen_helper={}
    # user_gen_helper={userID:[[ratings for genre1],[],[],...]}
    for id in user:
        # 初始化
        user_gen_helper[id]=[]
        for i in range(len(gen)):
            user_gen_helper[id].append([])
        # 遍历该用户看过的所有电影
        for mov_idx in range(len(user[id][0])):
            # 这部电影所属流派
            genOfmov=mov_gen[name[user[id][0][mov_idx]]]
            #  这部电影的评分
            score=user[id][1][mov_idx]
            for gen_idx in range(len(gen)):
                if gen[gen_idx] in genOfmov:
                     user_gen_helper[id][gen_idx].append(score)

    # 用户-流派矩阵，用户id对应行数
    user_gen=np.zeros([len(user)+1,len(gen_mov)])
    for id in user_gen_helper:
        for gen_idx in range(len(user_gen_helper[id])):
            if user_gen_helper[id][gen_idx]:
                user_gen[int(id)][gen_idx]=np.mean(user_gen_helper[id][gen_idx])
            else:
                user_gen[int(id)][gen_idx]=0
    return user_gen



In [2]:
import numpy as np

# 确认SVD应该保留几个特征值90%
# 计算能量
# Sigma升序排列
def enery(Sigma):
    whole_num=len(Sigma)
    Sig2=[]
    for i in Sigma:
        Sig2.append(i**2)
    whole_ener=sum(Sig2)
    for i in range(whole_num):
        if sum(Sig2[0:i])>=0.9*whole_ener:
            return i

# 对原始rating矩阵进行SVD分解
def svd(rating_martix):
    print('SVD分解...')
    U, Sigma, VT = np.linalg.svd(rating_martix)
    idx=np.argsort(Sigma)
    # 特征值排序
    idx=idx[::-1]
    k=enery(sorted(Sigma)[::-1])
    # get切片行列索引
    slice=idx[:k]
    U_new=U[:,slice]
    Sigma_new=np.diag((Sigma)[::-1][:k])
    VT_new=VT[slice,:]
    # 重构数据矩阵
    rate_new=np.dot(U_new,Sigma_new).dot(VT_new)
    return rate_new


# 向量相似度
# 欧式距离相似度
def ecludSim(A,B):
    return 1.0/(1.0 + np.linalg.norm(A - B))

# 余弦相似度
def cosSim(A,B):
    A_B = sum(A[i] * B[i] for i in range(len(A)))
    cos=A_B/(np.linalg.norm(A)*np.linalg.norm(B))
    return 0.5+0.5*cos

# 皮尔逊相似度（向量减去平均值后做余弦相似度）
def pearSim(A,B):
    meanA=np.mean(A)
    meanB = np.mean(B)
    A_B=sum((A[i]-meanA)*(B[i]-meanB) for i in range(len(A)))
    pear=A_B / (np.linalg.norm(A-meanA) * np.linalg.norm(B-meanB))
    return 0.5+0.5*pear


# 相似度矩阵
# 使用重构后的矩阵
def simMatrix(rate_new):
    sim=np.zeros([len(rate_new),len(rate_new)])
    for user1 in range(len(rate_new)):
        for user2 in range(len(rate_new)):
            # 用户和本身相似度设为负无穷
            if user1==user2:
                sim[user1][user2]=float('-inf')
            else:
                sim[user1][user2]=ecludSim(rate_new[user1],rate_new[user2])
    return sim

# 找到和目标用户最相似的K位用户
def closeUser(userID,sim,K):
    print('找到最相似的',K,'位用户...')
    simlist=list(sim[userID])
    res=[]
    while len(res)<K and max(simlist)!=float('-inf'):
        maxSim=max(simlist)
        idx=simlist.index(maxSim)
        res.append(idx)
        simlist[idx]=float('-inf')
    return res

# 找到推荐的k部电影
# user: {用户id:[[电影id],[评分]]}
def recomMovie(close_user_ID,user,k):
    print('找到推荐的电影列表...')
    all_movie,all_rate=[],[]
    for usr in close_user_ID:
        for movID,rate in zip(user[str(usr)][0],user[str(usr)][1]):
            all_movie.append(int(movID))
            all_rate.append(rate)
    idx=np.argsort(all_rate)[::-1]
    res=[]
    while len(res)<k:
        if all_movie[idx[0]] not in res:
            res.append(all_movie[idx[0]])
            idx=idx[1:]
    return res

def idToname(name,movID):
    res=[]
    for id in movID:
        res.append(name[str(id)])
    return res


def recommend(userID,closeUserNum,recommendMovieNum):
    name, mov_gen, gen_mov = loadGenre('movies.csv')
    rating, user = loadRating('ratings.csv', name)
    user_gen = user_genre(user, mov_gen, gen_mov, name)
    rating_new=svd(user_gen)
    sim=simMatrix(rating_new)
    close_user=closeUser(userID,sim,closeUserNum)
    recommendID=recomMovie(close_user,user,recommendMovieNum)
    return idToname(name,recommendID),recommendID,user


In [3]:
import numpy as np


def RMSE(predict,actual):
    """计算均方根误差
        RMSE加大了对预测不准的用户物品评分的惩罚（平方项的惩罚），因而对系统的评测更加苛刻
        @param self.predict, self.actual: 预测评价与真实评价记录的list
        @return: RMSE
    """
    error=sum(pow(pre-act,2) for pre,act in zip(predict,actual))
    return np.sqrt(error/len(predict))

def MSE(predict,actual):
    """计算平均绝对误差
        @param self.predict, self.actual: 预测评价与真实评价记录的list
        @return: MSE
    """
    error=sum(np.abs(pre-act) for pre,act in zip(predict,actual))
    return error/len(predict)

def precision(recommends, tests):
    """
    计算测试集用户的平均精确率：所有被检索到的item中,"应该被检索到"的item占的比例
    :param recommends: 预测出的推荐品字典{ userID : 推荐的物品 }
    :param tests: 真实的观看电影字典{ userID : 实际发生事务的物品 }
    """
    preci=0
    for user in recommends:
        pred=set(recommends[user])
        actual=set(tests[user])
        TP=len(pred & actual)
        preci+=TP/len(pred)
    return preci/len(recommends)

def recall(recommends, tests):
    """
    计算召回率：所有检索到的item占所有"应该检索到的item"的比例
    :param recommends: 预测出的推荐品字典{ userID : 推荐的物品 }
    :param tests: 真实的观看电影字典{ userID : 实际发生事务的物品 }
    """
    reca=0
    for user in recommends:
        pred=set(recommends[user])
        actual=set(tests[user])
        TP=len(pred & actual)
        reca+=TP/len(actual)
    return reca/len(recommends)

def coverage(recommends, all_items):
    """
        计算覆盖率: 推荐系统能够推荐出来的物品占总物品集合的比例
        @param recommends : dict形式 { userID : Items }
        @param all_items :  所有的电影，list
    """
    recom_item=set()
    for user in recommends:
        recom_item=recom_item | (set(recommends[user]) & set(all_items))
    return len(recom_item)/len(all_items)



In [4]:
import numpy as np
import random

# 随机挑选testSet
def selectTest(perc_test):
    print('随机挑选测试集...')
    # 提取某部电影的流派信息
    # name 收录所有电影名称{id:movie}
    # idx 对应电影ID
    # dic 为电影和其对应流派的字典
    name, mov_gen, gen_mov = loadGenre('movies.csv')
    # 返回{电影名称:[总分，[评分人id]]}
    # 返回{用户id:[[电影id],[评分]]}
    rating, user = loadRating('ratings.csv', name)
    # 确认训练集和测试集包含用户人数
    train_user_num=int(len(user)*(1-perc_test))
    test_user_num=len(user)-train_user_num
    # 随机挑选测试集
    test_user_ID=[]
    while len(test_user_ID)<test_user_num:
        id=str(random.randint(0,len(user)-1))
        if not id in test_user_ID:
            test_user_ID.append(id)
    user_test={}
    for id in test_user_ID:
        user_test[id]=user[id]
    # 生成训练集
    train_user_ID=[id for id in user if not id in test_user_ID]
    return test_user_ID,train_user_ID

test_user_ID,train_user_ID=selectTest(0.01)
# for k in [3,5,7,10]:
# 构造recommends, tests
recommends, tests={},{}
for testId in test_user_ID:
    print('计算第',testId,'位用户...')
    recomName,recomID,user=recommend(int(testId),3,5)
    testMovieID=user[str(testId)][0]
    recommends[testId]=recomID
    tests[testId]=[int(id) for id in testMovieID]
res=precision(recommends, tests)
print('precision is: ',round(res*100,2),'%')
rec=recall(recommends, tests)
print('recall is: ',round(rec*100,2),'%')

随机挑选测试集...
计算第 461 位用户...
SVD分解...
找到最相似的 3 位用户...
找到推荐的电影列表...
计算第 53 位用户...
SVD分解...
找到最相似的 3 位用户...
找到推荐的电影列表...
计算第 596 位用户...
SVD分解...
找到最相似的 3 位用户...
找到推荐的电影列表...
计算第 293 位用户...
SVD分解...
找到最相似的 3 位用户...
找到推荐的电影列表...
计算第 460 位用户...
SVD分解...
找到最相似的 3 位用户...
找到推荐的电影列表...
计算第 174 位用户...
SVD分解...
找到最相似的 3 位用户...
找到推荐的电影列表...
计算第 478 位用户...
SVD分解...
找到最相似的 3 位用户...
找到推荐的电影列表...
precision is:  22.86 %
recall is:  0.48 %
